In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

D:\anaconda3\envs\mfe-preprogram\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
connection_string = 'sqlite:///../../../Lectures/Lecture 4/data/data.db'

In [3]:
tokens = pd.read_sql('''
SELECT 
    token 
FROM
    ohlc
''', connection_string)

tokens = tokens['token'].unique().tolist()

tokens

['BTC',
 'ETH',
 'USDT',
 'SOL',
 'ADA',
 'DOT',
 'AVAX',
 'ATOM',
 'CRV',
 'AAVE',
 'COMP']

In [4]:
date_list = pd.read_sql('''
SELECT 
    ts
FROM 
    ohlc
''', connection_string)

date_list = date_list['ts'].tolist()
date_list = [i[:10] for i in date_list]
date_list = sorted(list(set(date_list)))
date_list = [datetime.strptime(i, '%Y-%m-%d') for i in date_list]
date_list[-17:]


[datetime.datetime(2021, 11, 29, 0, 0),
 datetime.datetime(2021, 11, 30, 0, 0),
 datetime.datetime(2021, 12, 1, 0, 0),
 datetime.datetime(2021, 12, 2, 0, 0),
 datetime.datetime(2021, 12, 3, 0, 0),
 datetime.datetime(2021, 12, 4, 0, 0),
 datetime.datetime(2021, 12, 5, 0, 0),
 datetime.datetime(2021, 12, 6, 0, 0),
 datetime.datetime(2021, 12, 7, 0, 0),
 datetime.datetime(2021, 12, 8, 0, 0),
 datetime.datetime(2021, 12, 9, 0, 0),
 datetime.datetime(2021, 12, 10, 0, 0),
 datetime.datetime(2021, 12, 11, 0, 0),
 datetime.datetime(2021, 12, 12, 0, 0),
 datetime.datetime(2021, 12, 13, 0, 0),
 datetime.datetime(2021, 12, 14, 0, 0),
 datetime.datetime(2021, 12, 15, 0, 0)]

In [5]:
[i.weekday() for i in date_list][-17:]

[0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2]

So we choose 2021-11-29 ~ 2021-12-13 as two complete weeks.

In [6]:
run_dates = [date_list[-17], date_list[-10]]
run_dates

[datetime.datetime(2021, 11, 29, 0, 0), datetime.datetime(2021, 12, 6, 0, 0)]

In [7]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)

In [8]:
for run_date in run_dates:
    run_date_str = str(run_date)[:10]
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7)}', end='\r')
        res = pm.execute_notebook(
            'market_analysis_template.ipynb',
            f'{base_output_folder}/market_analysis_{token}_{run_date_str}.ipynb',
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=7)}',
                'token': token,
                'connection_string': connection_string,
            }
        )

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

In [9]:
nb = sb.read_notebook('outputs/market_analysis_BTC_2021-11-29.ipynb')


In [12]:
nbs = sb.read_notebooks('outputs')

In [15]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'avg return': scrap['prices'].data['return'].mean(),
        'avg volatility': scrap['prices'].data['volatility'].mean(),
        'maximum drawdown': scrap['prices'].data['maxdrawdown'].min(), 
        'range of last close - first close': scrap['prices'].data['close'].values[-1] - scrap['prices'].data['close'].values[0],
               }) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,avg return,avg volatility,maximum drawdown,range of last close - first close
0,AAVE,2021-11-29 00:00:00,2021-12-06 00:00:00,-1.305232e-03,0.221054,-0.303471,-46.8080
1,AAVE,2021-12-06 00:00:00,2021-12-13 00:00:00,-3.755434e-04,0.200400,-0.181226,-11.6730
2,ADA,2021-11-29 00:00:00,2021-12-06 00:00:00,-9.344545e-04,0.226749,-0.250852,-0.2305
3,ADA,2021-12-06 00:00:00,2021-12-13 00:00:00,-1.302320e-04,0.203766,-0.183820,-0.0297
4,ATOM,2021-11-29 00:00:00,2021-12-06 00:00:00,-6.543502e-04,0.340796,-0.323731,-2.8370
5,ATOM,2021-12-06 00:00:00,2021-12-13 00:00:00,-9.716086e-07,0.254383,-0.145769,-0.0040
6,AVAX,2021-11-29 00:00:00,2021-12-06 00:00:00,-1.575971e-03,0.320661,-0.337174,-25.6000
7,AVAX,2021-12-06 00:00:00,2021-12-13 00:00:00,8.111556e-05,0.249173,-0.171316,1.1800
8,BTC,2021-11-29 00:00:00,2021-12-06 00:00:00,-8.976678e-04,0.175543,-0.213690,-7979.4100
9,BTC,2021-12-06 00:00:00,2021-12-13 00:00:00,9.997437e-05,0.134449,-0.092779,833.1100
